In [1]:
import numpy as np
import pandas as pd

#learning
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

#data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#preprocessing images
import cv2

## Preprocessing and loading in the data

In [2]:
# the values per image that belong to the given attributes for each class
attribute_values = np.load('feather-in-focus/attributes.npy')

# names of each bird class
labels = np.load("feather-in-focus/class_names.npy", allow_pickle=True).item()

# list of training data containing: image location, labels per image 
data = pd.read_csv("feather-in-focus/train_images.csv")


In [3]:
# code that adds the attributes per label to the training_df
att_list = []
for i in range(len(data)):
    att_list.append(attribute_values[data['label'][i]-1])  
    
data['attributes'] = att_list

In [4]:
def load_and_preprocess_images(image_paths, file_path):
    images = []
    for img_path in image_paths:
        img = cv2.imread(f'feather-in-focus/{file_path}{img_path}' )
        img = cv2.resize(img, (300, 300))  # Resize the image
        img = img / 255.0  # Normalize pixel values to [0, 1]
        images.append(img)
    return np.array(images)

In [6]:
# Load and preprocess images
X_images = load_and_preprocess_images(data['image_path'], 'train_images')
y_labels = to_categorical(data['label'].values-1)

X_train, X_test, y_train, y_test = train_test_split(X_images, y_labels, test_size=0.2, random_state=0)

## Using data augmentation to create more data

In [8]:
# Create an ImageDataGenerator with augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Apply data augmentation to your training data
augmented_data = datagen.flow(np.array(X_train), y_train, batch_size=32)

## Building the model


In [12]:
# Create Xception model with pre-trained weights
base_model = Xception(weights='imagenet', include_top=False, input_shape=(300, 300, 3), classes=200)

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers for classification
model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(y_train.shape[1], activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10  # Adjust as needed
model.fit(np.array(X_train), y_train, epochs=epochs, validation_data=(np.array(X_test), y_test))

# Evaluate the model
test_loss, test_acc = model.evaluate(np.array(X_test), y_test)
print(f'Test accuracy: {test_acc}')

Epoch 1/10
99/99 [==============================] - 154s 1s/step - loss: 4.8545 - accuracy: 0.0631 - val_loss: 4.0989 - val_accuracy: 0.1539
Epoch 2/10
99/99 [==============================] - 146s 1s/step - loss: 3.7225 - accuracy: 0.1780 - val_loss: 3.2153 - val_accuracy: 0.2837
Epoch 3/10
99/99 [==============================] - 147s 1s/step - loss: 3.0942 - accuracy: 0.2561 - val_loss: 2.7989 - val_accuracy: 0.3333
Epoch 4/10
99/99 [==============================] - 155s 2s/step - loss: 2.6894 - accuracy: 0.3274 - val_loss: 2.5033 - val_accuracy: 0.3957
Epoch 5/10
99/99 [==============================] - 147s 1s/step - loss: 2.3840 - accuracy: 0.3793 - val_loss: 2.2948 - val_accuracy: 0.4313
Epoch 6/10
99/99 [==============================] - 148s 1s/step - loss: 2.1379 - accuracy: 0.4274 - val_loss: 2.1553 - val_accuracy: 0.4326
Epoch 7/10
99/99 [==============================] - 142s 1s/step - loss: 1.9871 - accuracy: 0.4666 - val_loss: 2.1093 - val_accuracy: 0.4478
Epoch 8/10
99

In [13]:
# Unfreeze a few layers at a time
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Recompile the model after unfreezing layers
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Continue training
model.fit(np.array(X_train), y_train, epochs=5, validation_data=(np.array(X_test), y_test))


Epoch 1/5
99/99 [==============================] - 197s 2s/step - loss: 2.1005 - accuracy: 0.4213 - val_loss: 2.4675 - val_accuracy: 0.4160
Epoch 2/5
99/99 [==============================] - 187s 2s/step - loss: 1.4045 - accuracy: 0.5914 - val_loss: 2.1255 - val_accuracy: 0.4504
Epoch 3/5
99/99 [==============================] - 192s 2s/step - loss: 1.0023 - accuracy: 0.6850 - val_loss: 2.0672 - val_accuracy: 0.5127
Epoch 4/5
99/99 [==============================] - 186s 2s/step - loss: 0.7635 - accuracy: 0.7624 - val_loss: 1.9173 - val_accuracy: 0.5344
Epoch 5/5
99/99 [==============================] - 188s 2s/step - loss: 0.5819 - accuracy: 0.8166 - val_loss: 2.1106 - val_accuracy: 0.5216


## predicting the submission images and saving them 


In [14]:

test_df = pd.read_csv("feather-in-focus/test_images_path.csv")
test_images = load_and_preprocess_images(test_df['image_path'],'test_images')

# Predict
predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)


submission_df = pd.DataFrame({
    'id': test_df['id'],
    'label': predicted_labels+1
})
submission_df.to_csv('submission.csv', index=False)

125/125 [==============================] - 160s 1s/step
